In [ ]:
import sys
import os

sys.path.append(os.path.abspath(".."))
from pricers.mc_engine import simulate_gbm_paths
from pricers.monte_carlo_pricer import mc_pricer

import numpy as np


# Draft for an MC engine for exotics + comparison with BS

The problem : EULER BIAS 
The response : Monte carlo simulation 

Euler–Maruyama says “Over a very small time step Δt , just approximate the change.”
$$
S_{t+\Delta t}
=
S_t
+
\mu S_t \, \Delta t
+
\sigma S_t \sqrt{\Delta t}\, Z
$$

- It can go negative
- It introduces discretization bias

Hence MC simulation :


In [2]:


S = simulate_gbm_paths(S0=100, r=0.05, sigma=0.2, T=1, N=252, M=10)

print(S.shape)
import plotly.graph_objects as go

fig = go.Figure()
for i in range(S.shape[0]):
    fig.add_trace(go.Scatter(y=S[i], mode="lines", showlegend=False))

fig.show()


(10, 253)


### MC pricing is always for estimating something in the form of :
$$
V_0 = \mathbb{E}^{\mathbb{Q}}\!\left[e^{-rT} \cdot \text{Payoff}\right]
$$

#### What is the payoff function : 
Depends on:
- terminal price? european
- path average? asian (average the average)
- barrier hit? barrier option

In [3]:
def european_call_payoff(path, K):
    
    return max(path[-1] - K, 0.0) # here is done the selction of the terminal price of the path ( at T)

In [4]:


price, ci, se , beta= mc_pricer(
    payoff_fn=european_call_payoff,
    payoff_args=(100,),
    S0=100,
    r=0.05,
    sigma=0.2,
    T=1.0,
    N=1,
    M=100_000,
    seed=42
)
price , se

(np.float64(10.529223667713438), np.float64(0.046890875089576296))

In [5]:
price, ci, se , beta= mc_pricer(
    payoff_fn=european_call_payoff,
    payoff_args=(100,),
    S0=100,
    r=0.05,
    sigma=0.2,
    T=1.0,
    N=1,
    M=100_000,
    sim_method="antithetic",
    seed=42
)
price , se

(np.float64(10.446882902038286), np.float64(0.04648253464146022))

In [6]:
price, ci, se , beta= mc_pricer(
    payoff_fn=european_call_payoff,
    payoff_args=(100,),
    S0=100,
    r=0.05,
    sigma=0.2,
    T=1.0,
    N=1,
    M=100_000,
    sim_method="antithetic",
    use_control=True, 
    seed=42
)
price , se

(np.float64(10.482603833834027), np.float64(0.01779592365999398))

In [7]:
#asian call
def asian_call_payoff(path, K):
    avg_price = np.mean(path[1:])  # exclude S0 if desired
    return max(avg_price - K, 0.0)

price, ci, se, _ = mc_pricer(
    payoff_fn=asian_call_payoff,
    payoff_args=(100,),
    S0=100,
    r=0.05,
    sigma=0.2,
    T=1.0,
    N=252,
    M=100_000
)



In [8]:
def up_and_out_call_payoff(path, K, B):
    if np.max(path) >= B:
        return 0.0
    return max(path[-1] - K, 0.0)
price, ci, se, _ = mc_pricer(
    payoff_fn=up_and_out_call_payoff,
    payoff_args=(100, 130),
    S0=100,
    r=0.05,
    sigma=0.2,
    T=1.0,
    N=252,
    M=200_000
)

price, se 


(np.float64(3.5592640142789764), np.float64(0.014259677178530081))